# Compare cubic spline and GP with cubic polynomial

## Research

- [Bindel, Summer 2018 - Numerics for Data Science](./../literature/B-2018-numerics-for-data-science.pdf):
    - **Cubic splines and thin plate splines** At the end of the last lecture, we
    saw that we can write cubic splines for $1D$ function approximation in terms
    of the kernel $k(x, y) = |x - y|^3$. For functions on $\mathbb{R}^2$, the analogous choice is
    the thin plate spline with kernel $r \log r$ for $r = \|x - y\|$. Unlike the squared
    exponential and exponential kernels, we express these kernels in terms of the
    distance $r$ and not a scaled distance $r/l$, as approximation methods using
    cubic and thin plate splines are scale invariant: scaling the coordinate system
    does not change the predictions. Also unlike the squared exponential and
    exponential kernels, these kernels grow with increasing values of $r$, and so
    our intuition that kernels measure “how similar things are” founders a bit on
    these examples. Other ways of explaining kernels that are more appropriate
    to describing why these choices are useful.

- [Rasmussen & Williams, 2006 - Gaussian Processes for Machine Learning](./../literature/RW-2006-gaussian-processes-for-machine-learning.pdf):

    - Kernel: function mapping $x \in X , x' \in X$ into $\mathbb{R}$
    - Symmetric kernel: kernel with $k(x, x') = k(x', x)$
    - Covariance function: Symmetric psd kernel describing similarity in GPs
    - stationary covariance function: function of $x − x'$
    - isotropic covariance function aka radial basis functions (RBFs): function of $|x - x'|$
    - dot product covariance function: function of $x \cdot x'$

### Calculations

#### Gaussian process

The posterior mean of the Gaussian process is give by 

$
    \mu(x)=K(x,X)K(X,X)^{-1}y
$

where $X$ is the vector of $x$ coordinates of the known points ($(x_1,\dots,x_n)^\top$), $K$ is the Gram matrix with entries given by $K(X,X')_{ij}=k(x_i,x'_j), and $y$ is the vector of function values corresponding to the $x$ values in $X$ ($(f(x_1),\dots,f(x_n))^\top$).

We can add the gradients to the posterior mean for covariance functions that are symmetrical and twice symmetrically differentiable by settings

- $K(x,X)=(K_k(x,X), K_{\frac{\partial}{\partial b} k}(x, X))$
- $K(X,X)=\begin{pmatrix}K_k(X,X) & K_{\frac{\partial}{\partial b} k}(X,X)\\ K_{\frac{\partial}{\partial a} k}(X,X) & K_{\frac{\partial^2}{\partial a \partial b} k}(X,X)\end{pmatrix}$
- $y=(f(x_1)\dots,f(x_n),g(x_1),\dots,g(x_n))^\top$

where $K_f$ is the Gram matrix that results from using function $f$.

We can rewrite $z = K(X,X)^{-1}y$ as $z = K(X,X) \backslash y$ which is the solution to the linear system of the conditioned points.

We are interested in the Cubic kernel or Polyharmonic Spline Kernel with $p=3$. The polyharmonic spline is defined for odd $p$ as

$
    k(a,b) = |a-b|^p \\
    \frac{\partial}{\partial a} k(a,b) = p \cdot sign(a-b) \cdot |a-b|^{p-1} \\
    \frac{\partial}{\partial b} k(a,b) = -p \cdot sign(a-b) \cdot |a-b|^{p-1} \\
    \frac{\partial^2}{\partial a \partial b} k(a,b) = -(p^2 - p) \cdot |a-b|^{p-2}
$

##### Two points

Let us consider an example of two points, $x_i, f(x_i)=f_i, \frac{\partial}{\partial x}=g_i, i\in \{1,2\}$. WLOG we can assume that $x_1 < x_x$.

We have 

$
    \mu(x)=\begin{pmatrix}k(x,x_1) \\ k(x,x_2) \\ \frac{\partial}{\partial b}k(x,x_1) \\ \frac{\partial}{\partial b}k(x,x_2)\end{pmatrix}^\top z, 
    \quad z = \begin{pmatrix}
    k(x_1, x_1) & k(x_1,x_2) & \frac{\partial}{\partial b}k(x_1,x_1) & \frac{\partial}{\partial b}k(x_1,x_2) \\ 
    k(x_2,x_1) & k(x_2,x_2) & \frac{\partial}{\partial b}k(x_2,x_1) & \frac{\partial}{\partial b}k(x_2,x_2) \\ 
    \frac{\partial}{\partial a}k(x_1,x_1) & \frac{\partial}{\partial a}k(x_1,x_2) & \frac{\partial^2}{\partial a \partial b}k(x_1,x_1) & \frac{\partial^2}{\partial a \partial b}k(x_1,x_2) \\ 
    \frac{\partial}{\partial a}k(x_2,x_1) & \frac{\partial}{\partial a}k(x_2,x_2) & \frac{\partial^2}{\partial a \partial b}k(x_2,x_1) & \frac{\partial^2}{\partial a \partial b}k(x_2,x_2)
    \end{pmatrix} \backslash \begin{pmatrix}f_1 \\ f_2 \\ g_1 \\ g_2\end{pmatrix}
$

For the Polyharmonic Spline Kernels we have $k(x,x)=0, \frac{\partial}{\partial a}k(x,x)=0, \frac{\partial}{\partial b}k(x,x)=0, \frac{\partial^2}{\partial a \partial b}k(x,x)=0, k(a,b)=k(b,a), \frac{\partial}{\partial a}k(a,b)=-\frac{\partial}{\partial a}k(b,a), \frac{\partial}{\partial b}k(a,b)=-\frac{\partial}{\partial b}k(a,b), \frac{\partial^2}{\partial a \partial b}k(a,b)=\frac{\partial^2}{\partial a \partial b}k(b,a), \frac{\partial}{\partial a}k(a,b)=-\frac{\partial}{\partial b}k(a,b)$.

$
    \mu(x)=\begin{pmatrix}k(x,x_1) \\ k(x,x_2) \\ \frac{\partial}{\partial b}k(x,x_1) \\ \frac{\partial}{\partial b}k(x,x_2)\end{pmatrix}^\top z, 
    \quad z = \begin{pmatrix}
    0 & k(x_1,x_2) & 0 & -\frac{\partial}{\partial a}k(x_1,x_2) \\ 
    k(x_1,x_2) & 0 & \frac{\partial}{\partial a}k(x_1,x_2) & 0 \\ 
    0 & \frac{\partial}{\partial a}k(x_1,x_2) & 0 & \frac{\partial^2}{\partial a \partial b}k(x_1,x_2) \\ 
    -\frac{\partial}{\partial a}k(x_1,x_2) & 0 & \frac{\partial^2}{\partial a \partial b}k(x_1,x_2) & 0
    \end{pmatrix} \backslash \begin{pmatrix}f_1 \\ f_2 \\ g_1 \\ g_2\end{pmatrix}
$

We can now plug in the formula for the Polyharmonic Spline

$
    \mu(x)=\begin{pmatrix}|x-x_1|^p \\ |x-x_2|^p \\ -p \cdot sign(x-x_1) \cdot |x-x_1|^{p-1} \\ -p \cdot sign(x-x_2) \cdot |x-x_2|^{p-1}\end{pmatrix}^\top z,
    \quad z = \begin{pmatrix}
    0 & |x_1-x_2|^p & 0 & -p \cdot sign(x_1-x_2) \cdot |x_1-x_2|^{p-1} \\ 
    |x_1-x_2|^p & 0 & p \cdot sign(x_1-x_2) \cdot |x_1-x_2|^{p-1} & 0 \\ 
    0 & p \cdot sign(x_1-x_2) \cdot |x_1-x_2|^{p-1} & 0 & -(p^2 - p) \cdot |x_1-x_2|^{p-2} \\ 
    -p \cdot sign(x_1-x_2) \cdot |x_1-x_2|^{p-1} & 0 & -(p^2 - p) \cdot |x_1-x_2|^{p-2} & 0
    \end{pmatrix} \backslash \begin{pmatrix}f_1 \\ f_2 \\ g_1 \\ g_2\end{pmatrix}
$

Since $x_1 < x_2$, we know that $sign(x_1-x_2)=-1$

$
    \mu(x)=\begin{pmatrix}|x-x_1|^p \\ |x-x_2|^p \\ -p \cdot sign(x-x_1) \cdot |x-x_1|^{p-1} \\ -p \cdot sign(x-x_2) \cdot |x-x_2|^{p-1}\end{pmatrix}^\top z,
    \quad z = \begin{pmatrix}
    0 & |x_1-x_2|^p & 0 & p \cdot |x_1-x_2|^{p-1} \\ 
    |x_1-x_2|^p & 0 & -p \cdot |x_1-x_2|^{p-1} & 0 \\ 
    0 & -p \cdot |x_1-x_2|^{p-1} & 0 & -(p^2 - p) \cdot |x_1-x_2|^{p-2} \\ 
    p \cdot |x_1-x_2|^{p-1} & 0 & -(p^2 - p) \cdot |x_1-x_2|^{p-2} & 0
    \end{pmatrix} \backslash \begin{pmatrix}f_1 \\ f_2 \\ g_1 \\ g_2\end{pmatrix}
$

By defining $|x_1 - x_2|:=d$, we get

$
    \mu(x)=\begin{pmatrix}|x-x_1|^p \\ |x-x_2|^p \\ -p \cdot sign(x-x_1) \cdot |x-x_1|^{p-1} \\ -p \cdot sign(x-x_2) \cdot |x-x_2|^{p-1}\end{pmatrix}^\top z,
    \quad z = \begin{pmatrix}
    0 & d^p & 0 & p \cdot d^{p-1} \\ 
    d^p & 0 & -p \cdot d^{p-1} & 0 \\ 
    0 & -p \cdot d^{p-1} & 0 & -(p^2 - p) \cdot d^{p-2} \\ 
    p \cdot d^{p-1} & 0 & -(p^2 - p) \cdot d^{p-2} & 0
    \end{pmatrix} \backslash \begin{pmatrix}f_1 \\ f_2 \\ g_1 \\ g_2\end{pmatrix}
$

We can now solve the system

$
    \left[\hspace{-5pt}\begin{array}{cccc|c}
        0 
        & d^p 
        & 0 
        & p \cdot d^{p-1} 
        & f_1 
        \\ 
        d^p 
        & 0 
        & -p \cdot d^{p-1} 
        & 0 
        & f_2 
        \\
        0 
        & -p \cdot d^{p-1} 
        & 0 
        & -(p^2 - p) \cdot d^{p-2} 
        & g_1 
        \\
        p \cdot d^{p-1} 
        & 0 
        & -(p^2 - p) \cdot d^{p-2} 
        & 0 
        & g_2 
        \\
    \end{array}\hspace{-5pt}\right]
$

$L_4 = L_4 - \frac{p}{d} \cdot L_2$
$L_2 = \frac{1}{d^p} \cdot L_2$

$
    \left[\hspace{-5pt}\begin{array}{cccc|c}
        0 
        & d^p 
        & 0 
        & p \cdot d^{p-1} 
        & f_1 
        \\ 
        1
        & 0
        & - \frac{p}{d}
        & 0 
        & \frac{1}{d^p} \cdot f_2 
        \\
        0 
        & -p \cdot d^{p-1} 
        & 0 
        & -(p^2 - p) \cdot d^{p-2} 
        & g_1 
        \\
        0
        & 0 
        & p \cdot d^{p-2}
        & 0 
        & - \frac{p}{d} \cdot f_2 + g_2  
        \\
    \end{array}\hspace{-5pt}\right]
$

$L_1 = L_1 + \frac{d}{p} \cdot L_3$
$L_3 = -\frac{1}{p \cdot d^{p-1}} \cdot L_3$

$
    \left[\hspace{-5pt}\begin{array}{cccc|c}
        0 
        & 0
        & 0 
        & d^{p-1}
        & f_1 + \frac{d}{p} \cdot g_1 
        \\ 
        1
        & 0
        & - \frac{p}{d}
        & 0 
        & \frac{1}{d^p} \cdot f_2 
        \\
        0 
        & 1
        & 0 
        & \frac{(p - 1)}{d}
        & -\frac{1}{p \cdot d^{p-1}} \cdot g_1 
        \\
        0
        & 0 
        & p \cdot d^{p-2}
        & 0 
        & - \frac{p}{d} \cdot f_2 + g_2  
        \\
    \end{array}\hspace{-5pt}\right]
$

$L_2 = L_2 + \frac{1}{d^{p-1}} \cdot L_4$
$L_4 = \frac{1}{p \cdot d^{p-2}} \cdot L_4$

$
    \left[\hspace{-5pt}\begin{array}{cccc|c}
        0 
        & 0
        & 0 
        & d^{p-1}
        & f_1 + \frac{d}{p} \cdot g_1 
        \\ 
        1
        & 0
        & 0
        & 0 
        & -\frac{(p-1)}{d^{p}} \cdot f_2 + \frac{1}{d^{p-1}} \cdot g_2 
        \\
        0 
        & 1
        & 0 
        & \frac{(p - 1)}{d}
        & -\frac{1}{p \cdot d^{p-1}} \cdot g_1 
        \\
        0
        & 0 
        & 1
        & 0 
        & - \frac{1}{d^{p-1}} \cdot f_2 + \frac{1}{p \cdot d^{p-2}} \cdot g_2 
        \\
    \end{array}\hspace{-5pt}\right]
$

$L_3 = L_3 - \frac{(p-1)}{d^{p}} \cdot L_1$
$L_1 = \frac{1}{d^{p-1}} \cdot L_1$

$
    \left[\hspace{-5pt}\begin{array}{cccc|c}
        0 
        & 0
        & 0 
        & 1
        & \frac{1}{d^{p-1}} \cdot f_1 + \frac{1}{p \cdot d^{p-2}} \cdot g_1 
        \\ 
        1
        & 0
        & 0
        & 0 
        & -\frac{(p-1)}{d^{p}} \cdot f_2 + \frac{1}{d^{p-1}} \cdot g_2 
        \\
        0 
        & 1
        & 0 
        & 0
        & -\frac{(p-1)}{d^{p}} \cdot f_1 - \frac{1}{d^{p-1}} \cdot g_1
        \\
        0
        & 0 
        & 1
        & 0 
        & - \frac{1}{d^{p-1}} \cdot f_2 + \frac{1}{p \cdot d^{p-2}} \cdot g_2 
        \\
    \end{array}\hspace{-5pt}\right]
$

Solving the linear system, we get

$
    z=\begin{pmatrix}
        -\frac{(p-1)}{d^{p}} \cdot f_2 + \frac{1}{d^{p-1}} \cdot g_2 \\
        -\frac{(p-1)}{d^{p}} \cdot f_1 - \frac{1}{d^{p-1}} \cdot g_1 \\
        -\frac{1}{d^{p-1}} \cdot f_2 + \frac{1}{p \cdot d^{p-2}} \cdot g_2 \\
        \frac{1}{d^{p-1}} \cdot f_1 + \frac{1}{p \cdot d^{p-2}} \cdot g_1 \\
    \end{pmatrix}
$

Thus, our formula for $\mu$ is

$
    \mu(x)=
      |x-x_1|^p \cdot (-\frac{(p-1)}{d^{p}} \cdot f_2 + \frac{1}{d^{p-1}} \cdot g_2) \\
    + |x-x_2|^p \cdot (-\frac{(p-1)}{d^{p}} \cdot f_1 - \frac{1}{d^{p-1}} \cdot g_1) \\
    - p \cdot sign(x-x_1) \cdot |x-x_1|^{p-1} \cdot (-\frac{1}{d^{p-1}} \cdot f_2 + \frac{1}{p \cdot d^{p-2}} \cdot g_2) \\ 
    - p \cdot sign(x-x_2) \cdot |x-x_2|^{p-1} \cdot (\frac{1}{d^{p-1}} \cdot f_1 + \frac{1}{p \cdot d^{p-2}} \cdot g_1) \\
$

Which we can rearrange to

$
    \mu(x)=
    - |x-x_2|^{p-1} \cdot \frac{1}{d^{p-1}} \cdot (|x-x_2| \cdot \frac{(p-1)}{d} + p \cdot sign(x-x_2))  \cdot f_1 \\
    - |x-x_1|^{p-1} \cdot \frac{1}{d^{p-1}} \cdot (|x-x_1| \cdot \frac{(p-1)}{d} - p \cdot sign(x-x_1)) \cdot f_2 \\
    - |x-x_2|^{p-1} \cdot \frac{1}{d^{p-2}} \cdot (|x-x_2| \cdot \frac{1}{d} + sign(x-x_2)) \cdot g_1 \\
    + |x-x_1|^{p-1} \cdot \frac{1}{d^{p-2}} \cdot (|x-x_1| \cdot \frac{1}{d} - sign(x-x_1)) \cdot g_2 \\
$

For $p=3$ we get 

$
    \mu(x)=
    - |x-x_2|^{2} \cdot \frac{1}{d^{2}} \cdot (|x-x_2| \cdot \frac{2}{d} + 3 \cdot sign(x-x_2))  \cdot f_1 \\
    - |x-x_1|^{2} \cdot \frac{1}{d^{2}} \cdot (|x-x_1| \cdot \frac{2}{d} - 3 \cdot sign(x-x_1)) \cdot f_2 \\
    - |x-x_2|^{2} \cdot \frac{1}{d} \cdot (|x-x_2| \cdot \frac{1}{d} + sign(x-x_2)) \cdot g_1 \\
    + |x-x_1|^{2} \cdot \frac{1}{d} \cdot (|x-x_1| \cdot \frac{1}{d} - sign(x-x_1)) \cdot g_2 \\
$

Test at $x=x_1$ and $x=x_2$

$
    \mu(x_1) = f_1 \\
    \mu(x_2) = f_2 \\
$

##### Three points

Let us consider an example of three points, $x_i, f(x_i)=f_i, \frac{\partial}{\partial x}=g_i, i\in \{1,2,3\}$. WLOG we can assume that $x_1 < x_2 < x_3$.

We have:

$
    \mu(x)=K(x,(x_1,x_2, x_3)^\top)z, 
    \quad z = K((x_1,x_2,x_3)^\top,(x_1,x_2,x_3)^\top)\backslash (f_1, f_2, f_3, g_1, g_2, g_3)^\top\\
    \mu(x)=\begin{pmatrix}k(x,x_1) \\ k(x,x_2) \\ k(x,x_3) \\ \frac{\partial}{\partial a}k(x,x_1) \\ \frac{\partial}{\partial a}k(x,x_2) \\ \frac{\partial}{\partial a}k(x,x_3)\end{pmatrix}^\top z, 
    \quad z = \begin{pmatrix}
    k(x_1, x_1) & k(x_2,x_1) & k(x_3,x_1) & \frac{\partial}{\partial b}k(x_1,x_1) & \frac{\partial}{\partial b}k(x_1,x_2) & \frac{\partial}{\partial b}k(x_1,x_3) \\ 
    k(x_2,x_1) & k(x_2,x_2) & k(x_2,x_3) & \frac{\partial}{\partial b}k(x_2,x_1) & \frac{\partial}{\partial b}k(x_2,x_2) & \frac{\partial}{\partial b}k(x_2,x_3) \\ 
    k(x_3,x_1) & k(x_3,x_2) & k(x_3,x_3) & \frac{\partial}{\partial b}k(x_3,x_1) & \frac{\partial}{\partial b}k(x_3,x_2) & \frac{\partial}{\partial b}k(x_3,x_3) \\ 
    \frac{\partial}{\partial a}k(x_1,x_1) & \frac{\partial}{\partial a}k(x_1,x_2) & \frac{\partial}{\partial a}k(x_1,x_3) & \frac{\partial^2}{\partial a \partial b}k(x_1,x_1) & \frac{\partial^2}{\partial a \partial b}k(x_1,x_2) & \frac{\partial^2}{\partial a \partial b}k(x_1,x_3) 
    \\ \frac{\partial}{\partial a}k(x_2,x_1) & \frac{\partial}{\partial a}k(x_2,x_2) & \frac{\partial}{\partial a}k(x_2,x_3) & \frac{\partial^2}{\partial a \partial b}k(x_2,x_1) & \frac{\partial^2}{\partial a \partial b}k(x_2,x_2) & \frac{\partial^2}{\partial a \partial b}k(x_2,x_3) 
    \\ \frac{\partial}{\partial a}k(x_3,x_1) & \frac{\partial}{\partial a}k(x_3,x_2) & \frac{\partial}{\partial a}k(x_3,x_3) & \frac{\partial^2}{\partial a \partial b}k(x_3,x_1) & \frac{\partial^2}{\partial a \partial b}k(x_3,x_2) & \frac{\partial^2}{\partial a \partial b}k(x_3,x_3)
    \end{pmatrix} \backslash \begin{pmatrix}f_1 \\ f_2 \\ f_3 \\ g_1 \\ g_2 \\ g_3\end{pmatrix}
$

For the Polyharmonic Spline Kernels we have $k(x,x)=0, \frac{\partial}{\partial a}k(x,x)=0, \frac{\partial}{\partial b}k(x,x)=0, \frac{\partial^2}{\partial a \partial b}k(x,x)=0, k(a,b)=k(b,a), \frac{\partial}{\partial a}k(a,b)=-\frac{\partial}{\partial a}k(b,a), \frac{\partial}{\partial b}k(a,b)=-\frac{\partial}{\partial b}k(a,b), \frac{\partial^2}{\partial a \partial b}k(a,b)=\frac{\partial^2}{\partial a \partial b}k(b,a), \frac{\partial}{\partial a}k(a,b)=-\frac{\partial}{\partial b}k(a,b)$.

$
    \mu(x)=\begin{pmatrix}k(x,x_1) \\ k(x,x_2) \\ k(x,x_3) \\ \frac{\partial}{\partial a}k(x,x_1) \\ \frac{\partial}{\partial a}k(x,x_2) \\ \frac{\partial}{\partial a}k(x,x_3)\end{pmatrix}^\top z, 
    \quad z = \begin{pmatrix}
    0 & k(x_1,x_2) & k(x_1,x_3) & 0 & -\frac{\partial}{\partial a}k(x_1,x_2) & -\frac{\partial}{\partial a}k(x_1,x_3) \\ 
    k(x_1,x_2) & 0 & k(x_2,x_3) & \frac{\partial}{\partial a}k(x_1,x_2) & 0 & -\frac{\partial}{\partial a}k(x_2,x_3) \\ 
    k(x_1,x_3) & k(x_2,x_3) & 0 & \frac{\partial}{\partial a}k(x_1,x_3) & \frac{\partial}{\partial a}k(x_2,x_3) & 0 \\ 
    0 & \frac{\partial}{\partial a}k(x_1,x_2) & \frac{\partial}{\partial a}k(x_1,x_3) & 0 & \frac{\partial^2}{\partial a \partial b}k(x_1,x_2) & \frac{\partial^2}{\partial a \partial b}k(x_1,x_3) \\
    -\frac{\partial}{\partial a}k(x_1,x_2) & 0 & \frac{\partial}{\partial a}k(x_2,x_3) & \frac{\partial^2}{\partial a \partial b}k(x_1,x_2) & 0 & \frac{\partial^2}{\partial a \partial b}k(x_2,x_3) \\
    -\frac{\partial}{\partial a}k(x_1,x_3) & -\frac{\partial}{\partial a}k(x_2,x_3) & 0 & \frac{\partial^2}{\partial a \partial b}k(x_1,x_3) & \frac{\partial^2}{\partial a \partial b}k(x_2,x_3) & 0
    \end{pmatrix} \backslash \begin{pmatrix}f_1 \\ f_2 \\ f_3 \\ g_1 \\ g_2 \\ g_3\end{pmatrix}
$

We now substitute the Polyharmonic spline kernel

$
    \mu(x)=
    \begin{pmatrix}
        |x-x_1|^p \\ |x-x_2|^p \\ |x-x_3|^p \\ p \cdot sign(x-x_1) \cdot |x-x_1|^{p-1} \\ p \cdot sign(x-x_1) \cdot |x-x_1|^{p-1} \\ p \cdot sign(x-x_3) \cdot |x-x_3|^{p-1}
    \end{pmatrix}^\top z, 
    \quad z = 
    \left(\begin{matrix}
        0 & |x_1-x_2|^p & |x_1-x_3|^p \\
        |x_1-x_2|^p & 0 & |x_2-x_3|^p \\
        |x_1-x_3|^p & |x_2-x_3|^p & 0 \\
        0 & p \cdot sign(x_1-x_2) \cdot |x_1-x_2|^{p-1} & p \cdot sign(x_1-x_3) \cdot |x_1-x_3|^{p-1} \\
        -p \cdot sign(x_1-x_2) \cdot |x_1-x_2|^{p-1} & 0 & p \cdot sign(x_2-x_3) \cdot |x_2-x_3|^{p-1} \\
        -p \cdot sign(x_1-x_3) \cdot |x_1-x_3|^{p-1} & -p \cdot sign(x_2-x_3) \cdot |x_2-x_3|^{p-1} & 0 \\
    \end{matrix}\right.
    \\
    \qquad\left.\begin{matrix}
        0 & -p \cdot sign(x_1-x_2) \cdot |x_1-x_2|^{p-1} & -p \cdot sign(x_1-x_3) \cdot |x_1-x_3|^{p-1} \\ 
        p \cdot sign(x_1-x_2) \cdot |x_1-x_2|^{p-1} & 0 & -p \cdot sign(x_2-x_3) \cdot |x_2-x_3|^{p-1} \\ 
        p \cdot sign(x_1-x_3) \cdot |x_1-x_3|^{p-1} & p \cdot sign(x_2-x_3) \cdot |x_2-x_3|^{p-1} & 0 \\
        0 & -(p^2 - p) \cdot |x_1-x_2|^{p-2} & -(p^2 - p) \cdot |x_1-x_3|^{p-2} \\
        -(p^2 - p) \cdot |x_1-x_2|^{p-2} & 0 & -(p^2 - p) \cdot |x_2-x_3|^{p-2} \\
        -(p^2 - p) \cdot |x_1-x_3|^{p-2} & -(p^2 - p) \cdot |x_2-x_3|^{p-2} & 0
    \end{matrix}\right)
    \backslash \begin{pmatrix}f_1 \\ f_2 \\ f_3 \\ g_1 \\ g_2 \\ g_3\end{pmatrix}
$

Since we have an ordering of $x_1 < x_2 < x_3$ we can evaluate the $sign()$ expressions as $sign(x_1 - x_2)=-1, sign(x_1 - x_3)=-1, sign(x_2 - x_3)=-1$.

$
    \mu(x)=
    \begin{pmatrix}
        |x-x_1|^p \\ |x-x_2|^p \\ |x-x_3|^p \\ -p \cdot |x-x_1|^{p-1} \\ -p \cdot |x-x_1|^{p-1} \\ -p \cdot |x-x_3|^{p-1}
    \end{pmatrix}^\top z, 
    \quad z = 
    \left(\begin{matrix}
        0 & |x_1-x_2|^p & |x_1-x_3|^p &
        0 & p \cdot |x_1-x_2|^{p-1} & p \cdot |x_1-x_3|^{p-1} \\ 
        |x_1-x_2|^p & 0 & |x_2-x_3|^p &
        -p \cdot |x_1-x_2|^{p-1} & 0 & p \cdot |x_2-x_3|^{p-1} \\ 
        |x_1-x_3|^p & |x_2-x_3|^p & 0 &
        -p \cdot |x_1-x_3|^{p-1} & -p \cdot |x_2-x_3|^{p-1} & 0 \\
        0 & -p \cdot |x_1-x_2|^{p-1} & -p \cdot |x_1-x_3|^{p-1} &
        0 & -(p^2 - p) \cdot |x_1-x_2|^{p-2} & -(p^2 - p) \cdot |x_1-x_3|^{p-2} \\
        p \cdot |x_1-x_2|^{p-1} & 0 & -p \cdot |x_2-x_3|^{p-1} &
        -(p^2 - p) \cdot |x_1-x_2|^{p-2} & 0 & -(p^2 - p) \cdot |x_2-x_3|^{p-2} \\
        p \cdot |x_1-x_3|^{p-1} & p \cdot |x_2-x_3|^{p-1} & 0 &
        -(p^2 - p) \cdot |x_1-x_3|^{p-2} & -(p^2 - p) \cdot |x_2-x_3|^{p-2} & 0
    \end{matrix}\right)
    \backslash \begin{pmatrix}f_1 \\ f_2 \\ f_3 \\ g_1 \\ g_2 \\ g_3\end{pmatrix}
$

We define $|x_1 - x_2|:=d_1, |x_2 - x_3| = d_2$. Because of the ordering of $x_1 < x_2 < x_3$ we additionally get $|x_1 - x_3| = d_1 + d_2$.

$
    \mu(x)=
    \begin{pmatrix}
        |x-x_1|^p \\ |x-x_2|^p \\ |x-x_3|^p \\ -p \cdot |x-x_1|^{p-1} \\ -p \cdot |x-x_1|^{p-1} \\ -p \cdot |x-x_3|^{p-1}
    \end{pmatrix}^\top z, 
    \quad z = 
    \begin{pmatrix}
        0 & d_1^p & (d_1+d_2)^p &
        0 & p \cdot d_1^{p-1} & p \cdot (d_1+d_2)^{p-1} \\ 
        d_1^p & 0 & d_2^p &
        -p \cdot d_1^{p-1} & 0 & p \cdot d_2^{p-1} \\ 
        (d_1+d_2)^p & d_2^p & 0 &
        -p \cdot (d_1+d_2)^{p-1} & -p \cdot d_2^{p-1} & 0 \\
        0 & -p \cdot d_1^{p-1} & -p \cdot (d_1+d_2)^{p-1} &
        0 & -(p^2 - p) \cdot d_1^{p-2} & -(p^2 - p) \cdot (d_1+d_2)^{p-2} \\
        p \cdot d_1^{p-1} & 0 & -p \cdot d_2^{p-1} &
        -(p^2 - p) \cdot d_1^{p-2} & 0 & -(p^2 - p) \cdot d_2^{p-2} \\
        p \cdot (d_1+d_2)^{p-1} & p \cdot d_2^{p-1} & 0 &
        -(p^2 - p) \cdot (d_1+d_2)^{p-2} & -(p^2 - p) \cdot d_2^{p-2} & 0
    \end{pmatrix}
    \backslash \begin{pmatrix}f_1 \\ f_2 \\ f_3 \\ g_1 \\ g_2 \\ g_3\end{pmatrix}
$

We can now solve the system

$
    \left[\hspace{-5pt}\begin{array}{cccccc|c}
        0 
        & d_1^p 
        & (d_1+d_2)^p 
        & 0 & 
        p \cdot d_1^{p-1} 
        & p \cdot (d_1+d_2)^{p-1} 
        & f_1 
        \\ 
        d_1^p 
        & 0 
        & d_2^p 
        & -p \cdot d_1^{p-1} 
        & 0 
        & p \cdot d_2^{p-1} 
        & f_2 
        \\ 
        (d_1+d_2)^p 
        & d_2^p 
        & 0 
        & -p \cdot (d_1+d_2)^{p-1} 
        & -p \cdot d_2^{p-1} & 0 
        & f_3 
        \\
        0 
        & -p \cdot d_1^{p-1} 
        & -p \cdot (d_1+d_2)^{p-1} 
        & 0 & -(p^2 - p) \cdot d_1^{p-2} 
        & -(p^2 - p) \cdot (d_1+d_2)^{p-2} 
        & g_1 
        \\
        p \cdot d_1^{p-1} 
        & 0 
        & -p \cdot d_2^{p-1} 
        & -(p^2 - p) \cdot d_1^{p-2} 
        & 0 
        & -(p^2 - p) \cdot d_2^{p-2} 
        & g_2 
        \\
        p \cdot (d_1+d_2)^{p-1} 
        & p \cdot d_2^{p-1} 
        & 0 
        & -(p^2 - p) \cdot (d_1+d_2)^{p-2} 
        & -(p^2 - p) \cdot d_2^{p-2} 
        & 0 
        & g_3
    \end{array}\hspace{-5pt}\right]
$

$L_3 = L_3 - \frac{(d_1+d_2)^{p}}{d_1^{p}} \cdot L_2$,
$L_5 = L_5 - \frac{p}{d_1} \cdot L_2$, 
$L_6 = L_6 - \frac{p \cdot (d_1+d_2)^{p-1}}{d_1^{p}} \cdot L_2$,
$L_2 = \frac{1}{d_1^{p}} \cdot L_2$

$
    \left[\hspace{-5pt}\begin{array}{cccccc|c}
        0 
        & d_1^p 
        & (d_1+d_2)^p 
        & 0 & 
        p \cdot d_1^{p-1} 
        & p \cdot (d_1+d_2)^{p-1} 
        & f_1 
        \\ 
        1
        & 0
        & \frac{d_2^p}{d_1^{p}}
        & -p \cdot \frac{1}{d_1}
        & 0
        & p \cdot \frac{d_2^{p-1}}{d_1^{p}}
        & \frac{1}{d_1^{p}} \cdot f_2 
        \\ 
        0
        & d_2^p
        & -\frac{(d_1+d_2)^{p} \cdot d_2^p }{d_1^{p}} 
        & p \cdot (d_1+d_2)^{p-1} \cdot \frac{d_2}{d_1}
        & -p \cdot d_2^{p-1}
        & -p \cdot \frac{(d_1+d_2)^{p} \cdot d_2^{p-1}}{d_1^{p}}
        & f_3 - \frac{(d_1+d_2)^{p}}{d_1^{p}} \cdot f_2 
        \\
        0 
        & -p \cdot d_1^{p-1} 
        & -p \cdot (d_1+d_2)^{p-1} 
        & 0 & -(p^2 - p) \cdot d_1^{p-2} 
        & -(p^2 - p) \cdot (d_1+d_2)^{p-2} 
        & g_1 
        \\
        0
        & 0 
        & -p \cdot d_2^{p-1} \cdot (1 + \frac{d_2}{d_1})
        & p \cdot d_1^{p-2} 
        & 0 
        & p \cdot d_2^{p-1} \cdot ((1 - p) \cdot \frac{1}{d_2} - p \cdot \frac{1}{d_1})
        & g_2 - \frac{p}{d_1} \cdot f_2 
        \\ 
        0
        & p \cdot d_2^{p-1}
        & -p \cdot \frac{(d_1+d_2)^{p-1} \cdot d_2^p}{d_1^{p}}
        & p \cdot (d_1+d_2)^{p-2} \cdot (1 - p + p \cdot \frac{(d_1+d_2)}{d_1})
        & -(p^2 - p) \cdot d_2^{p-2}
        & -p^2 \cdot \frac{(d_1+d_2)^{p-1} \cdot d_2^{p-1} }{d_1^{p}}
        & g_3 - p \cdot \frac{(d_1+d_2)^{p-1}}{d_1^{p}} \cdot f_2 
        \\
    \end{array}\hspace{-5pt}\right]
$

$L_1 = L_1 - \frac{d_1^p}{d_2^p} \cdot L_3$,
$L_3 = \frac{1}{d_2^p} \cdot L_3$,
$L_4 = L_4 + \frac{p \cdot d_1^{p-1}}{d_2^p} \cdot L_3$,
$L_6 = L_6 - \frac{p}{d_2} \cdot L_3$,

$
    \left[\hspace{-5pt}\begin{array}{cccccc|c}
        0
        & 0
        & 2(d_1+d_2)^p
        & -p \cdot \frac{d_1^{p-1} \cdot (d_1+d_2)^{p-1}}{d_2^{p-1}}
        & p \cdot d_1^{p-1} \cdot (1 + \frac{d_1}{d_2})
        & p \cdot (d_1+d_2)^{p-1} \cdot \frac{d_1+2d_2}{d_2}
        & f_1 - \frac{d_1^p}{d_2^p} \cdot (f_3 - \frac{(d_1+d_2)^{p}}{d_1^{p}} \cdot f_2)
        \\ 
        1
        & 0
        & \frac{d_2^p}{d_1^{p}}
        & -p \cdot \frac{1}{d_1}
        & 0
        & p \cdot \frac{d_2^{p-1}}{d_1^{p}}
        & \frac{1}{d_1^{p}} \cdot f_2 
        \\ 
        0
        & 1
        & -\frac{(d_1+d_2)^{p}}{d_1^{p}}
        & p \cdot \frac{(d_1+d_2)^{p-1}}{d_2^{p-1} \cdot d_1} 
        & -p \cdot \frac{1}{d_2}
        & -p \cdot \frac{(d_1+d_2)^{p}}{d_2 \cdot d_1^{p}}
        & \frac{1}{d_2^p} \cdot (f_3 - \frac{(d_1+d_2)^{p}}{d_1^{p}} \cdot f_2)
        \\
        0 
        & 0
        & -p \cdot (d_1+d_2)^{p-1} \cdot \frac{2d_1+d_2}{d_1}
        & p^2 \cdot \frac{d_1^{p-2} \cdot (d_1+d_2)^{p-1}}{d_2^{p-1}}
        & -p \cdot d_1^{p-2} \cdot (p - 1 + p \cdot \frac{d_1}{d_2})
        & -p \cdot (d_1+d_2)^{p-2} \cdot (p - 1 + p \cdot \frac{(d_1+d_2)^2}{d_2\cdot d_1})
        & g_1 + \frac{p \cdot d_1^{p-1}}{d_2^p} \cdot (f_3 - \frac{(d_1+d_2)^{p}}{d_1^{p}} \cdot f_2)
        \\
        0
        & 0 
        & -p \cdot d_2^{p-1} \cdot \frac{d_1 + d_2}{d_1}
        & p \cdot d_1^{p-2} 
        & 0 
        & -p \cdot d_2^{p-1} \cdot ((p-1) \cdot \frac{1}{d_2} + p \cdot \frac{1}{d_1})
        & g_2 - \frac{p}{d_1} \cdot f_2 
        \\
        0
        & 0
        & p \cdot \frac{(d_1+d_2)^{p-1} \cdot d_2^{p-1} }{d_1^{p-1}}
        & -p \cdot (d_1+d_2)^{p-2} \cdot (p-1)
        & p \cdot d_2^{p-2}
        & p^2 \cdot \frac{(d_1+d_2)^{p-1} \cdot d_2^{p-2} }{d_1^{p-1}}
        & g_3 - p (\frac{(d_1+d_2)^{p-1}}{d_1^{p}} \cdot f_2 + (\frac{f_3}{d_2} - \frac{(d_1+d_2)^{p}}{d_1^{p} \cdot d_2} \cdot f_2))
        \\
    \end{array}\hspace{-5pt}\right]
$

$L_1 = L_1 + \frac{2(d_1+d_2)d_1}{p \cdot (2d_1+d_2)} \cdot L_4$,
$L_2 = L_2 + \frac{d_2^p}{p \cdot (d_1+d_2)^{p-1} \cdot d_1^{p-1} \cdot (2d_1+d_2)} \cdot L_4$,
$L_3 = L_3 - \frac{(d_1+d_2)}{p \cdot d_1^{p-1} \cdot (2d_1+d_2)} \cdot L_4$,
$L_4 = \frac{1}{-p \cdot (d_1+d_2)^{p-1} \cdot \frac{2d_1+d_2}{d_1}} \cdot L_4$,
$L_5 = L_5 - \frac{d_2^{p-1}}{(d_1+d_2)^{p-2} \cdot (2d_1+d_2)} \cdot L_4$,
$L_6 = L_6 + \frac{d_2^{p-1}}{(2d_1+d_2) \cdot d_1^{p-2}} \cdot L_4$

$
    \left[\hspace{-5pt}\begin{array}{cccccc|c}
        0
        & 0
        & 0
        & p \cdot \frac{d_1^{p-1} \cdot (d_1+d_2)^{p-1}}{d_2^{p-2}} \cdot \frac{1}{(2d_1+d_2)}
        & -d_1^{p-1} \cdot \frac{(p-2) \cdot (d_1+d_2)}{(2d_1+d_2)}
        & -(d_1+d_2)^{p-1} \cdot \frac{(p-2) \cdot d_1}{(2d_1+d_2)}
        & f_1 + \frac{2(d_1+d_2)d_1}{p \cdot (2d_1+d_2)} \cdot g_1
        \\ 
        1
        & 0
        & 0
        & -p \cdot \frac{2}{(2d_1+d_2)}
        & -\frac{d_2^{p-1}}{(d_1+d_2)^{p-1}} \cdot \frac{(pd_1-d_2+pd_2)}{d_1 \cdot (2d_1+d_2)}
        & d_2^{p-1} \cdot d_1^{p-1} \cdot (\frac{(pd_1+d_2)}{(2d_1^2+3d_1d_2+d_2^2)})
        & \frac{1}{d_1^{p-1} \cdot (2d_1+d_2)} \cdot f_2 + \frac{d_2^p}{p \cdot (d_1+d_2)^{p-1} \cdot d_1^{p-1} \cdot (2d_1+d_2)} \cdot g_1 + \frac{1}{(d_1+d_2)^{p-1} \cdot (2d_1+d_2)} \cdot f_3
        \\
        0
        & 1
        & 0
        & p \cdot \frac{(d_1+d_2)^{p-1}}{d_2^{p-1} \cdot (2d_1+d_2)}
        & \frac{(p-1) \cdot (d_1+d_2)}{d_1 \cdot (2d_1+d_2)}
        & -\frac{(d_1+d_2)^{p-1} \cdot (pd_1+d_2)}{d_2 \cdot d_1^{p-1} \cdot (2d_1+d_2)}
        & \frac{d_1}{d_2^p \cdot (2d_1+d_2)} \cdot f_3 +(\frac{1}{(2d_1+d_2)} - (d_1+d_2)) \cdot \frac{(d_1+d_2)^{p+1}}{d_1^{p} \cdot d_2^p} \cdot f_2 - \frac{(d_1+d_2)}{p \cdot d_1^{p-1} \cdot (2d_1+d_2)} \cdot g_1 
        \\
        \\
        0 
        & 0
        & -p \cdot (d_1+d_2)^{p-1} \cdot \frac{2d_1+d_2}{d_1}
        & p^2 \cdot \frac{d_1^{p-2} \cdot (d_1+d_2)^{p-1}}{d_2^{p-1}}
        & -p \cdot d_1^{p-2} \cdot (p - 1 + p \cdot \frac{d_1}{d_2})
        & -p \cdot (d_1+d_2)^{p-2} \cdot (p - 1 + p \cdot \frac{(d_1+d_2)^2}{d_2\cdot d_1})
        & g_1 + \frac{p \cdot d_1^{p-1}}{d_2^p} \cdot (f_3 - \frac{(d_1+d_2)^{p}}{d_1^{p}} \cdot f_2)
        \\
        0
        & 0 
        & -p \cdot d_2^{p-1} \cdot \frac{d_1 + d_2}{d_1}
        & p \cdot d_1^{p-2} 
        & 0 
        & -p \cdot d_2^{p-1} \cdot ((p-1) \cdot \frac{1}{d_2} + p \cdot \frac{1}{d_1})
        & g_2 - \frac{p}{d_1} \cdot f_2 
        \\
        0
        & 0
        & p \cdot \frac{(d_1+d_2)^{p-1} \cdot d_2^{p-1} }{d_1^{p-1}}
        & -p \cdot (d_1+d_2)^{p-2} \cdot (p-1)
        & p \cdot d_2^{p-2}
        & p^2 \cdot \frac{(d_1+d_2)^{p-1} \cdot d_2^{p-2} }{d_1^{p-1}}
        & g_3 - p (\frac{(d_1+d_2)^{p-1}}{d_1^{p}} \cdot f_2 + (\frac{f_3}{d_2} - \frac{(d_1+d_2)^{p}}{d_1^{p} \cdot d_2} \cdot f_2))
        \\
    \end{array}\hspace{-5pt}\right]
$

TODO: Continue at L5, L6, L4

#### Cubic Spline

The Hermite spline on the interval $[0,1]$ can be computed via

## Experiments

In [ ]:
from external.spline import Cubic, Spline
import matplotlib.pyplot as plt
import numpy as np
from gaussian_process import GaussianProcess
from kernels import CubicKernel
import GPfunctions as gp

In [ ]:
# Objective function


def fg0(x):
    return -x * np.sin(x), -x * np.cos(x) - np.sin(x)


def fg1(x):
    t_0 = np.sin(x)
    t_1 = np.abs(t_0)
    t_2 = x - 1
    t_3 = (1 + x) + np.abs(t_2)
    t_4 = np.cos(x)
    t_5 = np.exp(t_4)
    t_6 = x**3
    t_7 = t_5 + (t_6 * t_1)
    t_8 = t_3**2
    functionValue = t_7 / t_3
    gradient = (
        ((((3 * (x**2)) * t_1) / t_3) - ((t_5 * (t_0)) / t_3))
        + (((t_6 * t_4) * np.sign(t_0)) / t_3)
    ) - ((t_7 / t_8) + ((np.sign(t_2) * t_7) / t_8))

    return functionValue, gradient


fg = fg1

objectiveFunction = lambda x: fg(x)[0]
objectiveFunctionDerivative = lambda x: fg(x)[1]

X = np.linspace(start=-3.0, stop=3.0, num=1_000)
y, g = fg(X)

rng = np.random.default_rng(1)
training_indices = rng.choice(np.arange(y.size), size=6, replace=False)
# training_indices = rng.choice(np.arange(int(y.size / 4.0) ,int(y.size * (3.0 / 4.0))), size=6, replace=False)
training_indices.sort()
X_train, y_train, g_train = (
    X[training_indices],
    y[training_indices],
    g[training_indices],
)

In [ ]:
GP_posterior = GaussianProcess(CubicKernel(), X_train, y_train, g_train)

gp_mean, gp_variance = GP_posterior(X)

spline = Spline(fg)
for i in range(1, len(X_train)):
    spline.put(
        Cubic(
            X_train[i - 1],
            X_train[i],
            y_train[i - 1],
            g_train[i - 1],
            y_train[i],
            g_train[i],
        )
    )

fig, (ax1) = plt.subplots(1, 1)
gp.plot_objective(ax1, X, y, X_train, y_train)
gp.plot_gp(ax1, X, gp_mean, gp_variance, drawStd=False)
ax1.plot(X, [spline(x) for x in X], label="Spline")
gp.plot_label(ax1, "Spline vs kernel")
plt.show()

for x in X:
    if x in X_train:
        print(f"At x={x}:")
    print(f"delta={np.abs(spline(x) - GP_posterior(x)[0])}")